In [5]:
# !pip install transformers peft bitsandbytes accelerate trl pydantic-settings scipy codebleu

In [6]:
# DEFINE QUANTIZATION HERE. Choose from ("none" | "8bit" | "4bit")
QUANTIZATION = "4bit"#DO NOT CHANGE
DATASET = "codeparrot/xlcost-text-to-code" #DO NOT CHANGE
TESTSIZE = 200

In [7]:
#CHANGE THE FOLLOWING
EXPERT = "python" #python, cpp, java, javascript, etc.
MODELTYPE = "FINETUNED" #BASE or FINETUNED or MOE
model_id = "mistralai/Mistral-7B-v0.1"
loramodel = f"mistral-7b-{EXPERT}-LORA-4bit"
lorarepo = f'santoshsto/{loramodel}'
PARTITION = "Python-program-level" #java-program-level etc.

In [8]:
from huggingface_hub import login
login(token="")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

In [10]:
from datasets import load_dataset
data = load_dataset(DATASET, PARTITION)

In [11]:
next(iter(data["test"]))

{'text': 'Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array',
 'code': "def minSum ( A , N ) : NEW_LINE INDENT mp = { } NEW_LINE sum = 0 NEW_LINE for i in range ( N ) : NEW_LINE INDENT sum += A [ i ] NEW_LINE if A [ i ] in mp : NEW_LINE INDENT mp [ A [ i ] ] += 1 NEW_LINE DEDENT else : NEW_LINE INDENT mp [ A [ i ] ] = 1 NEW_LINE DEDENT DEDENT minSum = float ( ' inf ' ) NEW_LINE for it in mp : NEW_LINE INDENT minSum = min ( minSum , sum - ( it * mp [ it ] ) ) NEW_LINE DEDENT return minSum NEW_LINE DEDENT arr = [ 4 , 5 , 6 , 6 ] NEW_LINE N = len ( arr ) NEW_LINE print ( minSum ( arr , N ) ) NEW_LINE"}

In [12]:
train_dataset = data["train"]
eval_dataset = data["validation"]
test_dataset = data["test"]

In [13]:
print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'code'],
    num_rows: 9263
})
Dataset({
    features: ['text', 'code'],
    num_rows: 472
})
Dataset({
    features: ['text', 'code'],
    num_rows: 887
})


In [14]:
# Pre-define quantization configs

################## 4bit ##################
bb_config_4b_eval = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
##########################################

################## 8bit ##################
bb_config_8b_eval = BitsAndBytesConfig(
    load_in_8bit=True,
)
##########################################

def quantization_config_eval(quantization):
    if quantization == "8bit":
        return bb_config_8b_eval
    else:
        return bb_config_4b_eval

In [15]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
model_id = "mistralai/Mistral-7B-v0.1"
lora = "santoshsto/mistral-7b-python-LORA-4bit"
#model_id = "/home/ec2-user/remote-work/mistral-7b-python-FINETUNED-4bit"
if QUANTIZATION == "none":
    base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=False
    ).to("cuda")
else:
    base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config_eval(QUANTIZATION),
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=False
    )

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards: 100%|██████████| 2/2 [01:49<00:00, 54.66s/it]


In [16]:
model = PeftModel.from_pretrained(base_model, lora)
#model = model_to_merge.merge_and_unload()

In [17]:
from torch.nn.utils import parameters_to_vector

total_params = parameters_to_vector(model.parameters()).numel()
total_params

3922153472

In [18]:
eval_prompt = """Write a function to output the prime factorization of 2023 in python, C, and C++"""

In [19]:
eval_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array [/INST] \\n code:</s>"""

In [20]:
# eval_prompt = f"""<s>
# For a given integer n, print out all its prime factors one on each line. 
# n = 30
# [INST]
# """
model.eval()
with torch.no_grad():
  input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True).input_ids.cuda()
  outputs = model.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True, top_p=0.9,temperature=0.5)

  print(f"Prompt:\n{eval_prompt}\n")
  print(f"\nGenerated response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(eval_prompt):]}")
  print('''\nGround truth:\ndef print_prime_factors(n): 
    for i in range(2, n + 1):
      while n % i == 0:
        print(i)
        n //= i
  print_prime_factors(n)''')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Prompt:
<s>[INST] Write a python program to complete the following. Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array [/INST] \n code:</s>


Generated response:
 ] NEW_LINE def minSum ( arr , N ) : NEW_LINE INDENT mp = { } NEW_LINE for i in range ( N ) : NEW_LINE INDENT if arr [ i ] in mp : NEW_LINE INDENT mp [ arr [ i ] ] += 1 NEW_LINE DEDENT else : NEW_LINE INDENT mp [ arr [ i ] ] = 1 NEW_LINE DEDENT DEDENT sum = 0 NEW_LINE for i in range ( N ) : NEW_LINE INDENT sum += arr [ i ] NEW_LINE DEDENT minSum = sum NEW_LINE for key , value in mp . items ( ) : NEW_LINE INDENT minSum -= ( value * ( value - 1 ) // 2 ) NEW_LINE DEDENT return minSum NEW_LINE DEDENT arr = [ 1 , 1 , 2 , 2 ,

## Evaluation

In [21]:
import re
def extract_codeblock(response, prompt_length):
  delimiter = "code:"
 # Regular expression pattern to extract code block
  pattern = fr'{re.escape(delimiter)}(.*)'

  # Extract code block using regex
  match = re.search(pattern, response, re.DOTALL)

  if match:
      code_block = match.group(1)
      return code_block.strip()
  else:
      print("No code block found")
      return prediction[len(eval_prompt):]

In [22]:
from tqdm import tqdm
predictions = []
ground_truths = []
count = 0
pbar = tqdm(total=min(0, TESTSIZE), desc="Processing test dataset")

model.eval()
with torch.no_grad():
  for test_prompt in test_dataset:
    if count == TESTSIZE:
      break
    eval_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. {test_prompt['text']} [/INST] \\n </s>"""
    ground_truths.append(test_prompt['code'])
    input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True, top_p=0.9,temperature=0.5)
    prediction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    predictions.append(prediction[len(eval_prompt):])
    count += 1
    pbar.update(1)

pbar.close()

Processing test dataset: 0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing test dataset: 1it [00:56, 56.11s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing test dataset: 2it [01:52, 56.15s/it]The attention mask and 

In [23]:
import pandas as pd

# Create a DataFrame with a single column named "value"
predictions_df = pd.DataFrame({"value": predictions})

# Path to the output JSONL file
predictions_output_file = f"{MODELTYPE}_predictions_{EXPERT}.jsonl"

# Write the DataFrame to the JSONL file
predictions_df.to_json(predictions_output_file, orient='records', lines=True)

In [30]:
predictions_df.tail()

,value
195,"] def isKpalindrome ( S , k ) : NEW_LINE INDEN..."
196,"] def compare ( s1 , s2 ) : NEW_LINE INDENT s1..."
197,"] \n def printSteps ( s , n ) : NEW_LINE INDEN..."
198,"] \n def countFrequencies ( arr , n ) : NEW_LI..."
199,"] \n def checkDivisible ( binary , k ) : NEW_L..."


In [25]:
references_df = pd.DataFrame({"value": ground_truths})
references_output_file = f"{MODELTYPE}_ground_truths_{EXPERT}.jsonl"
references_df.to_json(references_output_file, orient='records', lines=True)

## CodeBleu

In [26]:
from codebleu import calc_codebleu
results = []
for idx, reference in enumerate(ground_truths):
  prediction = predictions[idx]
  result = calc_codebleu([reference], [prediction], lang=EXPERT, weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
  results.append(result)
print(result)
df = pd.DataFrame(results)
score_file = f'{MODELTYPE}_eval_report_{EXPERT}.csv'
df.to_csv(score_file)
# {
#   'codebleu': 0.5537,
#   'ngram_match_score': 0.1041,
#   'weighted_ngram_match_score': 0.1109,
#   'syntax_match_score': 1.0,
#   'dataflow_match_score': 1.0
# }

/opt/conda/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


{'codebleu': 0.5648788494616139, 'ngram_match_score': 0.3665529490690488, 'weighted_ngram_match_score': 0.6231211789361369, 'syntax_match_score': 0.5555555555555556, 'dataflow_match_score': 0.7142857142857143}


In [27]:
df['codebleu'].mean()

0.4061404234291351

In [28]:
report_df = pd.concat([references_df, predictions_df, df], axis="columns")
report_file = f'{MODELTYPE}_eval_full_report_{EXPERT}.csv'
report_df.to_csv(report_file)

In [29]:
# from google.colab import files
# files.download(score_file)
# files.download(predictions_output_file)
# files.download(references_output_file)
# files.download(report_file)